<a href="https://colab.research.google.com/github/SyedHarshath/Computer-Vision/blob/main/Background_Subtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import random

### Background Subtraction

In [ ]:
def compute_background(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray.astype("float"))
        count += 1
    cap.release()
    avg_background = np.mean(frames, axis=0).astype("uint8")
    return avg_background

In [ ]:
def subtract_background(video_path, background, threshold=30):
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, background)
        _, fg_mask = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        frame = cv2.resize(frame,(512,512))
        fg_mask = cv2.resize(fg_mask,(512,512))
        cv2.imshow('Original', frame)
        cv2.imshow('Foreground Mask', fg_mask)
        if cv2.waitKey(30) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

indoor_path = 'indoor.mp4'
outdoor_path = 'forest.mp4'
print("Computing Indoor Background...")
bg_indoor = compute_background(indoor_path)
print("Processing Indoor Video...")
subtract_background(indoor_path, bg_indoor)
print("Computing Outdoor Background...")
bg_outdoor = compute_background(outdoor_path)
print("Processing Outdoor Video...")
subtract_background(outdoor_path, bg_outdoor)

Computing Indoor Background...
Processing Indoor Video...
Computing Outdoor Background...
Processing Outdoor Video...


### Motion Tracking

In [ ]:
video_path = "op.mp4"
save_output = True
min_area = 1500
min_width, min_height = 30, 30
alpha = 0.03
frame_size = (512, 512)
colors = {}

In [ ]:
def get_color(object_id):
    if object_id not in colors:
        colors[object_id] = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
    return colors[object_id]

In [ ]:
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
frame = cv2.resize(frame, frame_size)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
background = gray.astype("float")

In [ ]:
if save_output:
    out_tracked = cv2.VideoWriter("tracked_output.avi", cv2.VideoWriter_fourcc(*"XVID"), 20, frame_size)
    out_fgmask = cv2.VideoWriter("foreground_output.avi", cv2.VideoWriter_fourcc(*"XVID"), 20, frame_size)

object_id = 0
centroids_prev = {}

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, frame_size)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.accumulateWeighted(gray, background, alpha)
    bg = cv2.convertScaleAbs(background)
    diff = cv2.absdiff(gray, bg)
    _, fg_mask = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    centroids_curr = {}
    for cnt in contours:
        if cv2.contourArea(cnt) < min_area:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        if w < min_width or h < min_height:
            continue
        centroid = (int(x + w / 2), int(y + h / 2))
        matched_id = None
        for cid, prev_centroid in centroids_prev.items():
            if abs(prev_centroid[0] - centroid[0]) < 30 and abs(prev_centroid[1] - centroid[1]) < 30:
                matched_id = cid
                break
        if matched_id is None:
            matched_id = object_id
            object_id += 1
        centroids_curr[matched_id] = centroid
        color = get_color(matched_id)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"ID {matched_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.circle(frame, centroid, 4, color, -1)
    centroids_prev = centroids_curr.copy()
    fg_mask_color = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)
    cv2.imshow("Tracked Objects", frame)
    cv2.imshow("Foreground", fg_mask)
    if save_output:
        out_tracked.write(frame)
        out_fgmask.write(fg_mask_color)
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
if save_output:
    out_tracked.release()
    out_fgmask.release()
cv2.destroyAllWindows()

### Scene Change Estimation

In [ ]:
def compute_background(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray.astype("float"))
        count += 1
    cap.release()
    avg_background = np.mean(frames, axis=0).astype("uint8")
    return avg_background

In [ ]:
def estimate_scene_threshold(video_path, threshold_pixel=25, sample_count=10):
    cap = cv2.VideoCapture(video_path)
    diffs = []
    ret, frame1 = cap.read()
    if not ret:
        cap.release()
        return 0.05
    prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    for _ in range(sample_count):
        ret, frame2 = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, prev)
        ratio = np.sum(diff > threshold_pixel) / diff.size
        diffs.append(ratio)
        prev = gray
    cap.release()
    return np.mean(diffs) + 2 * np.std(diffs)

In [ ]:
def subtract_background(video_path, background, threshold=30, scene_threshold=0.05, pixel_thresh=25, dynamic_bg=False):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    if not ret:
        return
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_index = 1
    background = background.astype("float32")
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    alpha = 0.01
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (512, 512))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff_scene = cv2.absdiff(gray, prev_gray)
        scene_change_ratio = np.sum(diff_scene > pixel_thresh) / diff_scene.size
        if scene_change_ratio > scene_threshold:
            print(f"[Scene Change] Frame: {frame_index} | Change: {scene_change_ratio * 100:.2f}%")
            prev_gray = gray.copy()
            cv2.imshow('Original', frame_resized)
            cv2.imshow('Foreground Mask', diff_scene)
            if cv2.waitKey(30) & 0xFF == 27:
                break
            frame_index += 1
            continue
        diff = cv2.absdiff(gray, background.astype("uint8"))
        _, fg_mask = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_DILATE, kernel)
        fg_mask = cv2.resize(fg_mask, (512, 512))
        cv2.imshow('Original', frame_resized)
        cv2.imshow('Foreground Mask', fg_mask)
        if dynamic_bg:
            background = cv2.addWeighted(gray.astype("float32"), alpha, background, 1 - alpha, 0)
        if cv2.waitKey(30) & 0xFF == 27:
            break
        prev_gray = gray.copy()
        frame_index += 1
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
path = 'steyn.mp4'
bg = compute_background(path)
print("Estimating Scene Threshold...")
scene_thresh = estimate_scene_threshold(path)
subtract_background(path, bg, scene_threshold=scene_thresh, dynamic_bg=True)

Estimating Scene Threshold...
[Scene Change] Frame: 13 | Change: 55.58%
[Scene Change] Frame: 80 | Change: 53.76%
[Scene Change] Frame: 150 | Change: 52.78%
[Scene Change] Frame: 206 | Change: 47.06%
[Scene Change] Frame: 269 | Change: 51.29%
[Scene Change] Frame: 339 | Change: 52.43%
[Scene Change] Frame: 396 | Change: 56.66%
[Scene Change] Frame: 482 | Change: 48.20%
[Scene Change] Frame: 540 | Change: 47.01%
